In [1]:
# !pip install pyrankvote
# !pip install pandas

In [13]:
import pyrankvote
import ast
import pandas as pd
from pyrankvote import Candidate, Ballot


def get_cand_list(df):
    list_of_unique_vals_in_dataframe = df.iloc[:, 1].value_counts().index
    cand_list = list(filter(lambda x: x != '0', list_of_unique_vals_in_dataframe))
    return cand_list

def initialize_cand_objs(cand_list):
    if isinstance(cand_list, str):
        return [Candidate(c) for c in cand_list.split(', ')]
    else:
        return [Candidate(c) for c in cand_list]

def initialize_cand_objs_in_df_cols(df):
    cands_df = df.iloc[:, 1:]  # getting all columns with candidate names in them
    cands_df = cands_df.applymap(lambda x: create_cand(x))  # making them into pyrankvote candidate objects
    cands_df['ballot_id'] = df.iloc[:, 0]  # stitching df back with ballot_ids from old dataframe
    return cands_df

def get_cands_into_single_cell(df):
    df['candidate_list'] = df.iloc[:, 1:].agg(", ".join, axis=1) #this excludes the first candidates??
    return df

def initialize_ballot_objs(df):
    ballot_objects = []
    for index, value in enumerate(df['candidate_list']):
        ballot = Ballot(ranked_candidates=value)
        ballot_objects.append(ballot)
    return ballot_objects

def run_election(list_of_cand_objs, election_df):
    return pyrankvote.instant_runoff_voting(list_of_cand_objs, election_df['ballots'], pick_random_if_blank=True)

def rm_invalid_rows(df):
    return df[df['candidate_list'] != '0']


def main(df):
    
    # creating dictionary with including winners, rounds and election number:
    election_dict = {}
    
    #renaming column to ballot_id
    df.rename(columns={'Unnamed: 0': 'ballot_id'}, inplace=True)
    
    #droping nans, 
    df = df.dropna(axis=1, how='all')
    df = df.drop(['ballot_id','filename'], axis=1)
    print('df.shape',df.shape)
        
    #aggregating all cand into one cell
    df = get_cands_into_single_cell(df)
    
    #Removing 0 with empty space:
    df['candidate_list'] = df['candidate_list'].apply(lambda x: x.replace('0, ', '').replace(', 0', ''))
    
    #Removing invalid rows:
    df = rm_invalid_rows(df)
    
    #for all srings making into pyrank candidate objects:
    df['candidate_list'] = df['candidate_list'].apply(lambda x: initialize_cand_objs(x))

    #creating ballot objects column for each candidate in candidate list:
    ballots = initialize_ballot_objs(df)
    df['ballots'] = ballots

    #getting unique list of candidates:
    cand_list = get_cand_list(df)
    cand_list = initialize_cand_objs(cand_list)
    print("cand_list: ",cand_list)
    
    #getting  number of candidates:
#     num_candidates=df['number_candidates']
#     print("number of candidates in election",num_candidates)
#     election_dict['num_candidates'] = num_candidates
    
    
    #running pyrank election and getting winners:
    election = run_election(cand_list, df)
    winner = election.get_winners()
    
    #Dictionary keys and values *************************
    #printing winners and add dictionary key:
    print("get_winners",winner)
    election_dict['winner'] = winner
    
    # printing rounds and add dictionary key:
    print("rounds",len(election.rounds))
    election_dict['rounds'] = len(election.rounds)
    
    #print elections and add dictionary key:
    print("election:", election)
    election_dict['election'] = election

        
    #will return df and dictionary for us to read votes
    return df, election_dict


In [17]:
df = pd.read_csv('master_elections.csv')
df.rename(columns={'Unnamed: 0': 'ballot_id'}, inplace=True)
df = df[(df['filename']!="election_05-04-2020_08-39-40_5cands_12noise.csv") | (df['ballot_id']!=2767)]
df = df[(df['filename']!="election_04-30-2020_21-07-57_4_0.01.csv") | (df['ballot_id']!=20726)]
df = df[(df['filename']!="election_05-04-2020_08-40-23_5cands_16noise.csv") | (df['ballot_id']!=41363)]
master_df = df.copy(deep=True)


election_ids = df.filename.unique()

election_num_dict = {}
for election_num, election in enumerate(election_ids):
    election_num_dict[election_num] = election

In [18]:
def read_votes(file, election_dict):
    with open(file, 'r') as file1:
        count = 0
        while True: 
            count += 1

            # Get next line from file 
            line = file1.readline()
            if not line: 
                break
            strip_line = line.strip()
            
            if 'ROUND' in strip_line or "FINAL" in strip_line:
                start_round = True
                if "ROUND 1" in strip_line:
                    monitor_round1 = True
                else:
                    monitor_round1 = False
                if "ROUND 2" in strip_line:
                    monitor_round2 = True
                else:
                    monitor_round2 = False
                if "ROUND 3" in strip_line:
                    monitor_round3 = True
                else:
                    monitor_round3 = False
                if "ROUND 4" in strip_line:
                    monitor_round4 = True
                else:
                    monitor_round4 = False
                if "FINAL" in strip_line:
                    final_round =True
                else:
                    final_round=False
                
            if 'ROUND' not in strip_line and 'FINAL' not in strip_line:
                start_round = False
                
                if 'candidate_1' in line and monitor_round1:
                    if 'can1firstRoundVotes' not in election_dict:
                        election_dict['can1firstRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                if 'candidate_1' in line and monitor_round2:
                    if 'can1secondRoundVotes' not in election_dict:
                        election_dict['can1secondRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                if 'candidate_1' in line and monitor_round3:
                    if 'can1thirdRoundVotes' not in election_dict:
                        election_dict['can1thirdRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                if 'candidate_1' in line and monitor_round4:
                    if 'can1fourthRoundVotes' not in election_dict:
                        election_dict['can1fourthRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()  
                if 'candidate_1' in line and final_round:
                    if 'can1finalRoundVotes' not in election_dict:
                        election_dict['can1finalRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()      
                        
                if 'candidate_2' in line and monitor_round1:
                    if 'can2firstRoundVotes' not in election_dict:
                        election_dict['can2firstRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                if 'candidate_2' in line and monitor_round2:
                    if 'can2secondRoundVotes' not in election_dict:
                        election_dict['can2secondRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                if 'candidate_2' in line and monitor_round3:
                    if 'can2thirdRoundVotes' not in election_dict:
                        election_dict['can2thirdRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                if 'candidate_2' in line and monitor_round4:
                    if 'can2fourthRoundVotes' not in election_dict:
                        election_dict['can2fourthRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()  
                if 'candidate_2' in line and final_round:
                    if 'can2finalRoundVotes' not in election_dict:
                        election_dict['can2finalRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()    
                
                if 'candidate_3' in line and monitor_round1:
                    if 'can3firstRoundVotes' not in election_dict:
                        election_dict['can3firstRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                if 'candidate_3' in line and monitor_round2:
                    if 'can3secondRoundVotes' not in election_dict:
                        election_dict['can3secondRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                if 'candidate_3' in line and monitor_round3:
                    if 'can3thirdRoundVotes' not in election_dict:
                        election_dict['can3thirdRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                if 'candidate_3' in line and monitor_round4:
                    if 'can3fourthRoundVotes' not in election_dict:
                        election_dict['can3fourthRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()  
                if 'candidate_3' in line and final_round:
                    if 'can3finalRoundVotes' not in election_dict:
                        election_dict['can3finalRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()    
                
                if 'candidate_4' in line and monitor_round1:
                    if 'can4firstRoundVotes' not in election_dict:
                        election_dict['can4firstRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                if 'candidate_4' in line and monitor_round2:
                    if 'can4secondRoundVotes' not in election_dict:
                        election_dict['can4secondRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                if 'candidate_4' in line and monitor_round3:
                    if 'can4thirdRoundVotes' not in election_dict:
                        election_dict['can4thirdRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                if 'candidate_4' in line and monitor_round4:
                    if 'can4fourthRoundVotes' not in election_dict:
                        election_dict['can4fourthRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()  
                if 'candidate_4' in line and final_round:
                    if 'can4finalRoundVotes' not in election_dict:
                        election_dict['can4finalRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()    
                    
                #Winner round: 
                if winner in line and monitor_round1:
                    if 'winnerfirstRoundVotes' not in election_dict:
                        election_dict['winnerfirstRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                if winner in line and monitor_round2:
                    if 'winnersecondRoundVotes' not in election_dict:
                        election_dict['winnersecondRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                if winner in line and monitor_round3:
                    if 'winnerthirdRoundVotes' not in election_dict:
                        election_dict['winnerthirdRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                if winner in line and monitor_round4:
                    if 'winnerfourthRoundVotes' not in election_dict:
                        election_dict['winnerfourthRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()  
                if winner in line and final_round:
                    if 'winnerfinalRoundVotes' not in election_dict:
                        election_dict['winnerfinalRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()                       
                    
        file1.close()               
                    

In [19]:
#building dictionary
all_elect_dict={}


#Enumerating through the election and election_num in the election_id array and applying main function while printing:
for election_num, election in enumerate(election_ids):
    
    print(election_num, election,' \n ')
    print("***********Election: ", election_num)
    df_election = df.loc[df['filename']==election]
    
 
    temp_df,election_dict = main(df_election)
    print(election_dict)

    
    if len(election_dict['winner'])==1:
        winner = election_dict['winner'][0].name
        election_dict['winner'] = winner
    else:
        print(election, "has more than one winner")
    result_file = 'out.txt'
    
    with open(result_file, 'w') as file1:
        print(election_dict['election'], file=file1)
    read_votes(result_file, election_dict) #<<<==== where we actually call read_votes
    all_elect_dict[election_num] = election_dict
    

0 election_05-04-2020_08-40-23_5cands_16noise.csv  
 
***********Election:  0
df.shape (41363, 5)
cand_list:  [<Candidate('candidate_1')>, <Candidate('candidate_5')>, <Candidate('candidate_2')>, <Candidate('candidate_3')>, <Candidate('candidate_4')>]
get_winners [<Candidate('candidate_1')>]
rounds 4
election: ROUND 1
Candidate      Votes  Status
-----------  -------  --------
candidate_1     9129  Hopeful
candidate_5     9010  Hopeful
candidate_2     8455  Hopeful
candidate_3     7911  Hopeful
candidate_4     6858  Rejected

ROUND 2
Candidate      Votes  Status
-----------  -------  --------
candidate_1    10902  Hopeful
candidate_5    10641  Hopeful
candidate_3    10078  Hopeful
candidate_2     9742  Rejected
candidate_4        0  Rejected

ROUND 3
Candidate      Votes  Status
-----------  -------  --------
candidate_1    14073  Hopeful
candidate_3    13673  Hopeful
candidate_5    13617  Rejected
candidate_2        0  Rejected
candidate_4        0  Rejected

FINAL RESULT
Candidate    

In [ ]:
all_elect_dict

In [7]:
election_results_df=pd.DataFrame()
election_results_list=[]

In [8]:
for key,val in all_elect_dict.items():
    val['election_id']=election_num_dict[key]
    print(val)

    election_results_list.append(val)

{'winner': 'candidate_1', 'rounds': 4, 'election': <ElectionResults(4 rounds)>, 'can1firstRoundVotes': 9129, 'winnerfirstRoundVotes': 9129, 'can2firstRoundVotes': 8455, 'can3firstRoundVotes': 7911, 'can4firstRoundVotes': 6858, 'can1secondRoundVotes': 10902, 'winnersecondRoundVotes': 10902, 'can3secondRoundVotes': 10078, 'can2secondRoundVotes': 9742, 'can4secondRoundVotes': 0, 'can1thirdRoundVotes': 14070, 'winnerthirdRoundVotes': 14070, 'can3thirdRoundVotes': 13673, 'can2thirdRoundVotes': 0, 'can4thirdRoundVotes': 0, 'can1finalRoundVotes': 20704, 'winnerfinalRoundVotes': 20704, 'can3finalRoundVotes': 20659, 'can2finalRoundVotes': 0, 'can4finalRoundVotes': 0, 'election_id': 'election_05-04-2020_08-40-23_5cands_16noise.csv'}
{'winner': 'candidate_1', 'rounds': 3, 'election': <ElectionResults(3 rounds)>, 'can1firstRoundVotes': 13918, 'winnerfirstRoundVotes': 13918, 'can3firstRoundVotes': 12693, 'can2firstRoundVotes': 10097, 'can4firstRoundVotes': 7022, 'can1secondRoundVotes': 16899, 'winn

In [9]:
election_results_df = pd.DataFrame(election_results_list)
election_results_df.to_csv('election_results.csv', index=False)

In [10]:
election_results_df.head()

,winner,rounds,election,can1firstRoundVotes,winnerfirstRoundVotes,can2firstRoundVotes,can3firstRoundVotes,can4firstRoundVotes,can1secondRoundVotes,winnersecondRoundVotes,...,winnerfinalRoundVotes,can3finalRoundVotes,can2finalRoundVotes,can4finalRoundVotes,election_id,can3fourthRoundVotes,winnerfourthRoundVotes,can2fourthRoundVotes,can1fourthRoundVotes,can4fourthRoundVotes
0,candidate_1,4,ROUND 1\nCandidate Votes Status\n-------...,9129,9129,8455,7911,6858.0,10902.0,10902.0,...,20704,20659,0,0.0,election_05-04-2020_08-40-23_5cands_16noise.csv,NaN,NaN,NaN,NaN,NaN
1,candidate_1,3,ROUND 1\nCandidate Votes Status\n-------...,13918,13918,10097,12693,7022.0,16899.0,16899.0,...,22952,20778,0,0.0,election_04-30-2020_21-00-03_4_0.0225.csv,NaN,NaN,NaN,NaN,NaN
2,candidate_4,4,ROUND 1\nCandidate Votes Status\n-------...,7177,7502,6848,6384,7502.0,8735.0,9065.0,...,17359,0,0,17359.0,election_05-02-2020_08-18-17_5cands_3noise.csv,NaN,NaN,NaN,NaN,NaN
3,candidate_1,3,ROUND 1\nCandidate Votes Status\n-------...,3400,3400,2269,2053,2343.0,4475.0,4475.0,...,5757,0,0,4308.0,election_05-04-2020_08-38-34_4cands_12noise.csv,NaN,NaN,NaN,NaN,NaN
4,candidate_2,3,ROUND 1\nCandidate Votes Status\n-------...,11210,11236,11236,10110,9061.0,14785.0,15603.0,...,24977,0,24977,0.0,election_04-30-2020_20-59-34_4_0.0375.csv,NaN,NaN,NaN,NaN,NaN


In [11]:
# of votes per candidate in total / total votes
# of rounds in election*

# In total, how many 1st-place votes did the winner get
# In total, how many last-place votes did the 

In [12]:
# def get_total_candidates(election_results_df):
    
#     election_results_df.fillna(0, inplace=True)
#     df = election_results_df 
    
#     df['Can1Total'] = df.apply(lambda row: row['can1firstRoundVotes'] + row['can1secondRoundVotes']+ row['can1thirdRoundVotes'] + row['can1fourthRoundVotes'] + row['can1finalRoundVotes'], axis=1)
#     df['Can2Total'] = df.apply(lambda row: row['can2firstRoundVotes'] + row['can2secondRoundVotes']+ row['can2thirdRoundVotes'] + row['can2fourthRoundVotes'] + row['can2finalRoundVotes'], axis=1)
#     df['Can3Total'] = df.apply(lambda row: row['can3firstRoundVotes'] + row['can3secondRoundVotes']+ row['can3thirdRoundVotes'] + row['can3fourthRoundVotes'] + row['can3finalRoundVotes'], axis=1)
#     df['Can4Total'] = df.apply(lambda row: row['can4firstRoundVotes'] + row['can4secondRoundVotes']+ row['can4thirdRoundVotes'] + row['can4fourthRoundVotes'] + row['can4finalRoundVotes'], axis=1)
    
#     df['first_rouns_list'] = df[['can1firstRoundVotes','can2firstRoundVotes', 'can3firstRoundVotes','can4firstRoundVotes']].values.tolist()
#     df['sec_rouns_list'] = df[['can1secondRoundVotes', 	'can2secondRoundVotes', 'can3secondRoundVotes', 'can4secondRoundVotes']].values.tolist()
#     df['third_rouns_list'] = df[['can1thirdRoundVotes','can2thirdRoundVotes','can3thirdRoundVotes','can4thirdRoundVotes']].values.tolist()
#     df['fourth_rouns_list'] = df[['can1fourthRoundVotes','can2fourthRoundVotes', 'can3fourthRoundVotes', 'can4fourthRoundVotes']].values.tolist()
#     df['final_rouns_list'] = df[['can1finalRoundVotes','can2finalRoundVotes', 'can3finalRoundVotes', 'can4finalRoundVotes']].values.tolist()
    
#     return df

In [13]:
election_results_df.fillna(0, inplace=True)

In [14]:
df = election_results_df 

In [15]:
df.to_csv("df.csv")

In [16]:
import pandas as pd
df = pd.read_csv('df.csv')

In [17]:
df.head()

,Unnamed: 0,winner,rounds,election,can1firstRoundVotes,winnerfirstRoundVotes,can2firstRoundVotes,can3firstRoundVotes,can4firstRoundVotes,can1secondRoundVotes,...,winnerfinalRoundVotes,can3finalRoundVotes,can2finalRoundVotes,can4finalRoundVotes,election_id,can3fourthRoundVotes,winnerfourthRoundVotes,can2fourthRoundVotes,can1fourthRoundVotes,can4fourthRoundVotes
0,0,candidate_1,4,ROUND 1\nCandidate Votes Status\n-------...,9129,9129,8455,7911,6858.0,10902.0,...,20704,20659,0,0.0,election_05-04-2020_08-40-23_5cands_16noise.csv,0.0,0.0,0.0,0.0,0.0
1,1,candidate_1,3,ROUND 1\nCandidate Votes Status\n-------...,13918,13918,10097,12693,7022.0,16899.0,...,22952,20778,0,0.0,election_04-30-2020_21-00-03_4_0.0225.csv,0.0,0.0,0.0,0.0,0.0
2,2,candidate_4,4,ROUND 1\nCandidate Votes Status\n-------...,7177,7502,6848,6384,7502.0,8735.0,...,17359,0,0,17359.0,election_05-02-2020_08-18-17_5cands_3noise.csv,0.0,0.0,0.0,0.0,0.0
3,3,candidate_1,3,ROUND 1\nCandidate Votes Status\n-------...,3400,3400,2269,2053,2343.0,4475.0,...,5757,0,0,4308.0,election_05-04-2020_08-38-34_4cands_12noise.csv,0.0,0.0,0.0,0.0,0.0
4,4,candidate_2,3,ROUND 1\nCandidate Votes Status\n-------...,11210,11236,11236,10110,9061.0,14785.0,...,24977,0,24977,0.0,election_04-30-2020_20-59-34_4_0.0375.csv,0.0,0.0,0.0,0.0,0.0


In [18]:
# df['TotalVotesWinner'] = df['firstRoundVotes']+df['secondRoundVotes']+df['thirdRoundVotes']+df['fourthRoundVotes']+df['finalRoundVotes']

In [19]:
#Getting total number of votes for each candidate in each round in each election


df['Can1Total'] = df.apply(lambda row: row['can1firstRoundVotes'] + row['can1secondRoundVotes']+ row['can1thirdRoundVotes'] + row['can1fourthRoundVotes'] + row['can1finalRoundVotes'], axis=1)

df['Can2Total'] = df.apply(lambda row: row['can2firstRoundVotes'] + row['can2secondRoundVotes']+ row['can2thirdRoundVotes'] + row['can2fourthRoundVotes'] + row['can2finalRoundVotes'], axis=1)

df['Can3Total'] = df.apply(lambda row: row['can3firstRoundVotes'] + row['can3secondRoundVotes']+ row['can3thirdRoundVotes'] + row['can3fourthRoundVotes'] + row['can3finalRoundVotes'], axis=1)

df['Can4Total'] = df.apply(lambda row: row['can4firstRoundVotes'] + row['can4secondRoundVotes']+ row['can4thirdRoundVotes'] + row['can4fourthRoundVotes'] + row['can4finalRoundVotes'], axis=1)

In [20]:
# df['Can5Total'] = df.apply(lambda row: row['can5firstRoundVotes'] + row['can5secondRoundVotes']+ row['can5thirdRoundVotes'] + row['can5fourthRoundVotes'] + row['can5finalRoundVotes'], axis=1)

In [21]:
# df.drop(columns=['firstRoundVotes','secondRoundVotes','thirdRoundVotes','finalRoundVotes','fourthRoundVotes'],inplace=True)

In [22]:
df.head()

,Unnamed: 0,winner,rounds,election,can1firstRoundVotes,winnerfirstRoundVotes,can2firstRoundVotes,can3firstRoundVotes,can4firstRoundVotes,can1secondRoundVotes,...,election_id,can3fourthRoundVotes,winnerfourthRoundVotes,can2fourthRoundVotes,can1fourthRoundVotes,can4fourthRoundVotes,Can1Total,Can2Total,Can3Total,Can4Total
0,0,candidate_1,4,ROUND 1\nCandidate Votes Status\n-------...,9129,9129,8455,7911,6858.0,10902.0,...,election_05-04-2020_08-40-23_5cands_16noise.csv,0.0,0.0,0.0,0.0,0.0,54805.0,18197.0,52321.0,6858.0
1,1,candidate_1,3,ROUND 1\nCandidate Votes Status\n-------...,13918,13918,10097,12693,7022.0,16899.0,...,election_04-30-2020_21-00-03_4_0.0225.csv,0.0,0.0,0.0,0.0,0.0,53769.0,21249.0,49150.0,7022.0
2,2,candidate_4,4,ROUND 1\nCandidate Votes Status\n-------...,7177,7502,6848,6384,7502.0,8735.0,...,election_05-02-2020_08-18-17_5cands_3noise.csv,0.0,0.0,0.0,0.0,0.0,43094.0,26114.0,13710.0,45448.0
3,3,candidate_1,3,ROUND 1\nCandidate Votes Status\n-------...,3400,3400,2269,2053,2343.0,4475.0,...,election_05-04-2020_08-38-34_4cands_12noise.csv,0.0,0.0,0.0,0.0,0.0,13632.0,4962.0,2053.0,9548.0
4,4,candidate_2,3,ROUND 1\nCandidate Votes Status\n-------...,11210,11236,11236,10110,9061.0,14785.0,...,election_04-30-2020_20-59-34_4_0.0375.csv,0.0,0.0,0.0,0.0,0.0,42635.0,51816.0,21339.0,9061.0


In [23]:
df.columns

Index(['Unnamed: 0', 'winner', 'rounds', 'election', 'can1firstRoundVotes',
       'winnerfirstRoundVotes', 'can2firstRoundVotes', 'can3firstRoundVotes',
       'can4firstRoundVotes', 'can1secondRoundVotes', 'winnersecondRoundVotes',
       'can3secondRoundVotes', 'can2secondRoundVotes', 'can4secondRoundVotes',
       'can1thirdRoundVotes', 'winnerthirdRoundVotes', 'can3thirdRoundVotes',
       'can2thirdRoundVotes', 'can4thirdRoundVotes', 'can1finalRoundVotes',
       'winnerfinalRoundVotes', 'can3finalRoundVotes', 'can2finalRoundVotes',
       'can4finalRoundVotes', 'election_id', 'can3fourthRoundVotes',
       'winnerfourthRoundVotes', 'can2fourthRoundVotes',
       'can1fourthRoundVotes', 'can4fourthRoundVotes', 'Can1Total',
       'Can2Total', 'Can3Total', 'Can4Total'],
      dtype='object')

In [24]:
df['first_rouns_list'] = df[['can1firstRoundVotes', 	'can2firstRoundVotes', 	'can3firstRoundVotes', 	'can4firstRoundVotes']].values.tolist()
df['sec_rouns_list'] = df[['can1secondRoundVotes', 	'can2secondRoundVotes', 	'can3secondRoundVotes', 'can4secondRoundVotes']].values.tolist()
df['third_rouns_list'] = df[['can1thirdRoundVotes', 	'can2thirdRoundVotes', 	'can3thirdRoundVotes', 	'can4thirdRoundVotes']].values.tolist()
df['fourth_rouns_list'] = df[['can1fourthRoundVotes', 	'can2fourthRoundVotes', 'can3fourthRoundVotes', 'can4fourthRoundVotes']].values.tolist()
df['final_rouns_list'] = df[['can1finalRoundVotes', 	'can2finalRoundVotes', 'can3finalRoundVotes', 'can4finalRoundVotes']].values.tolist()

In [25]:
df.head()

,Unnamed: 0,winner,rounds,election,can1firstRoundVotes,winnerfirstRoundVotes,can2firstRoundVotes,can3firstRoundVotes,can4firstRoundVotes,can1secondRoundVotes,...,can4fourthRoundVotes,Can1Total,Can2Total,Can3Total,Can4Total,first_rouns_list,sec_rouns_list,third_rouns_list,fourth_rouns_list,final_rouns_list
0,0,candidate_1,4,ROUND 1\nCandidate Votes Status\n-------...,9129,9129,8455,7911,6858.0,10902.0,...,0.0,54805.0,18197.0,52321.0,6858.0,"[9129.0, 8455.0, 7911.0, 6858.0]","[10902.0, 9742.0, 10078.0, 0.0]","[14070.0, 0.0, 13673.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[20704.0, 0.0, 20659.0, 0.0]"
1,1,candidate_1,3,ROUND 1\nCandidate Votes Status\n-------...,13918,13918,10097,12693,7022.0,16899.0,...,0.0,53769.0,21249.0,49150.0,7022.0,"[13918.0, 10097.0, 12693.0, 7022.0]","[16899.0, 11152.0, 15679.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[22952.0, 0.0, 20778.0, 0.0]"
2,2,candidate_4,4,ROUND 1\nCandidate Votes Status\n-------...,7177,7502,6848,6384,7502.0,8735.0,...,0.0,43094.0,26114.0,13710.0,45448.0,"[7177.0, 6848.0, 6384.0, 7502.0]","[8735.0, 8359.0, 7326.0, 9065.0]","[11056.0, 10907.0, 0.0, 11522.0]","[0.0, 0.0, 0.0, 0.0]","[16126.0, 0.0, 0.0, 17359.0]"
3,3,candidate_1,3,ROUND 1\nCandidate Votes Status\n-------...,3400,3400,2269,2053,2343.0,4475.0,...,0.0,13632.0,4962.0,2053.0,9548.0,"[3400.0, 2269.0, 2053.0, 2343.0]","[4475.0, 2693.0, 0.0, 2897.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[5757.0, 0.0, 0.0, 4308.0]"
4,4,candidate_2,3,ROUND 1\nCandidate Votes Status\n-------...,11210,11236,11236,10110,9061.0,14785.0,...,0.0,42635.0,51816.0,21339.0,9061.0,"[11210.0, 11236.0, 10110.0, 9061.0]","[14785.0, 15603.0, 11229.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[16640.0, 24977.0, 0.0, 0.0]"


In [26]:
import numpy as np

In [27]:
df.winnerfirstRoundVotes=df.winnerfirstRoundVotes.astype(float)
df.winnersecondRoundVotes=df.winnersecondRoundVotes.astype(float)
df.winnerthirdRoundVotes=df.winnerthirdRoundVotes.astype(float)
df.winnerfourthRoundVotes=df.winnerfourthRoundVotes.astype(float)
df.winnerfinalRoundVotes=df.winnerfinalRoundVotes.astype(float)

df['winner_1st_place_votes_r1'] = np.where(df.winnerfirstRoundVotes == df.first_rouns_list.map(lambda x: x[0]),
                                           df.winnerfirstRoundVotes, 0)

df['winner_1st_place_votes_r2'] = np.where(df.winnersecondRoundVotes == df.sec_rouns_list.map(lambda x: x[0]),
                                           df.winnersecondRoundVotes, 0)

df['winner_1st_place_votes_r3'] = np.where(df.winnerthirdRoundVotes == df.third_rouns_list.map(lambda x: x[0]),
                                           df.winnerthirdRoundVotes, 0)

df['winner_1st_place_votes_r4'] = np.where(df.winnerfourthRoundVotes == df.fourth_rouns_list.map(lambda x: x[0]),
                                           df.winnerfourthRoundVotes, 0)

# df['winner_1st_place_votes_fr'] = np.where(df.winnerfinalRoundVotes == df.final_rouns_list.map(lambda x: x[0]), #since it is obvious that the first place vote would go to the winner should we include final ?
#                                            df.winnerfinalRoundVotes, 0)

df['total_1st_place_winner_votes'] = df['winner_1st_place_votes_r1'] + df['winner_1st_place_votes_r2'] + df['winner_1st_place_votes_r3'] + df['winner_1st_place_votes_r4']

In [28]:
df['winner_last_place_votes_r1'] = np.where(df.winnerfirstRoundVotes == df.first_rouns_list.map(lambda x: x[-1]),
                                           df.winnerfirstRoundVotes, 0)

df['winner_last_place_votes_r2'] = np.where(df.winnersecondRoundVotes == df.sec_rouns_list.map(lambda x: x[-1]),
                                           df.winnersecondRoundVotes, 0)

df['winner_last_place_votes_r3'] = np.where(df.winnerthirdRoundVotes == df.third_rouns_list.map(lambda x: x[-1]),
                                           df.winnerthirdRoundVotes, 0)

df['winner_last_place_votes_r4'] = np.where(df.winnerfourthRoundVotes == df.fourth_rouns_list.map(lambda x: x[-1]),
                                           df.winnerfourthRoundVotes, 0)

# df['winner_1st_place_votes_fr'] = np.where(df.winnerfinalRoundVotes == df.final_rouns_list.map(lambda x: x[0]), #since it is obvious that the first place vote would go to the winner should we include final ?
#                                            df.winnerfinalRoundVotes, 0)

df['total_last_place_winner_votes'] = df['winner_last_place_votes_r1'] + df['winner_last_place_votes_r2'] + df['winner_last_place_votes_r3'] + df['winner_last_place_votes_r4']

In [29]:
df.head()

,Unnamed: 0,winner,rounds,election,can1firstRoundVotes,winnerfirstRoundVotes,can2firstRoundVotes,can3firstRoundVotes,can4firstRoundVotes,can1secondRoundVotes,...,winner_1st_place_votes_r1,winner_1st_place_votes_r2,winner_1st_place_votes_r3,winner_1st_place_votes_r4,total_1st_place_winner_votes,winner_last_place_votes_r1,winner_last_place_votes_r2,winner_last_place_votes_r3,winner_last_place_votes_r4,total_last_place_winner_votes
0,0,candidate_1,4,ROUND 1\nCandidate Votes Status\n-------...,9129,9129.0,8455,7911,6858.0,10902.0,...,9129.0,10902.0,14070.0,0.0,34101.0,0.0,0.0,0.0,0.0,0.0
1,1,candidate_1,3,ROUND 1\nCandidate Votes Status\n-------...,13918,13918.0,10097,12693,7022.0,16899.0,...,13918.0,16899.0,0.0,0.0,30817.0,0.0,0.0,0.0,0.0,0.0
2,2,candidate_4,4,ROUND 1\nCandidate Votes Status\n-------...,7177,7502.0,6848,6384,7502.0,8735.0,...,0.0,0.0,0.0,0.0,0.0,7502.0,9065.0,11522.0,0.0,28089.0
3,3,candidate_1,3,ROUND 1\nCandidate Votes Status\n-------...,3400,3400.0,2269,2053,2343.0,4475.0,...,3400.0,4475.0,0.0,0.0,7875.0,0.0,0.0,0.0,0.0,0.0
4,4,candidate_2,3,ROUND 1\nCandidate Votes Status\n-------...,11210,11236.0,11236,10110,9061.0,14785.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Get the max value in each list
# if value is == to winner's value for specific round
# count value as 1st place vote for winner

# for last place do this same thing


In [31]:
df_final = df[[ 'winner', 'rounds', 'election', 'Can1Total', 'Can2Total','Can3Total','Can4Total', 'total_1st_place_winner_votes','total_last_place_winner_votes']]

In [32]:
df_final.head()

,winner,rounds,election,Can1Total,Can2Total,Can3Total,Can4Total,total_1st_place_winner_votes,total_last_place_winner_votes
0,candidate_1,4,ROUND 1\nCandidate Votes Status\n-------...,54805.0,18197.0,52321.0,6858.0,34101.0,0.0
1,candidate_1,3,ROUND 1\nCandidate Votes Status\n-------...,53769.0,21249.0,49150.0,7022.0,30817.0,0.0
2,candidate_4,4,ROUND 1\nCandidate Votes Status\n-------...,43094.0,26114.0,13710.0,45448.0,0.0,28089.0
3,candidate_1,3,ROUND 1\nCandidate Votes Status\n-------...,13632.0,4962.0,2053.0,9548.0,7875.0,0.0
4,candidate_2,3,ROUND 1\nCandidate Votes Status\n-------...,42635.0,51816.0,21339.0,9061.0,0.0,0.0


In [33]:
df_final.to_csv("df_final.csv")